# **FEATURE SELECTION**

In [ ]:
import numpy as np
import random
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Dataset
X, y = load_breast_cancer(return_X_y=True)
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.2, random_state=42)

n_features = X.shape[1]
print("Número de características:", n_features)


Número de características: 30


In [ ]:
# Cromosoma: vector binario de longitud n_features
# Generar población inicial
def random_chrom():
    return [random.choice([0,1]) for _ in range(n_features)]

#Evalúa la calidad de un cromosoma
def fitness(chrom):
    mask = np.array(chrom, dtype=bool)
    if mask.sum() == 0:  # evita seleccionar 0 columnas
        return 0
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(Xtr[:,mask], ytr)
    return clf.score(Xval[:,mask], yval)
#Seleccionar los mejores
def tournament_selection(pop, fitnesses, k=3):
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(range(len(pop))), k)
        winner = max(aspirants, key=lambda i: fitnesses[i])
        selected.append(pop[winner])
    return selected
# Cruzar pares de padres
def one_point_crossover(p1, p2):
    point = random.randint(1, n_features-1)
    return p1[:point]+p2[point:], p2[:point]+p1[point:]
#Mutar ligeramente
def mutate(chrom, pm=0.1):
    for i in range(len(chrom)):
        if random.random() < pm:
            chrom[i] = 1 - chrom[i]
    return chrom


In [ ]:
POP_SIZE = 6 # Cromosomas
GENERATIONS = 5

# Inicialización
pop = [random_chrom() for _ in range(POP_SIZE)]
print("Población inicial:", pop)

fitnesses = [fitness(ch) for ch in pop]

# Evolución
for gen in range(GENERATIONS):
    print(f"\n=== Generación {gen} ===")
    for i, (chrom, fit) in enumerate(zip(pop, fitnesses)):
        print(f" Individuo {i}: {chrom} → Fitness={fit:.4f}")

    parents = tournament_selection(pop, fitnesses, k=3)

    next_pop = []
    for i in range(0, POP_SIZE, 2):
        c1, c2 = one_point_crossover(parents[i], parents[(i+1)%POP_SIZE])
        next_pop += [c1, c2]

    next_pop = [mutate(c) for c in next_pop]
    print("   Después de mutación:", next_pop)

    fitnesses = [fitness(ch) for ch in next_pop]
    pop = next_pop

# Mejor solución
best_idx = np.argmax(fitnesses)
print("\n=== Mejor solución encontrada ===")
print("Cromosoma:", pop[best_idx])
print("Fitness:", fitnesses[best_idx])
print("Características seleccionadas:", np.where(np.array(pop[best_idx])==1)[0])


Población inicial: [[1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1]]

=== Generación 0 ===
 Individuo 0: [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1] → Fitness=0.9561
 Individuo 1: [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1] → Fitness=0.9298
 Individuo 2: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0] → Fitness=0.9386
 Individuo 3: [1, 0, 0, 0, 1, 1, 0, 0, 0

# **HYPERPARAMETER OPTIMIZATION**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X, y = load_iris(return_X_y=True)
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import random

def random_chrom():
    return [
        random.randint(10, 200),   # número de árboles
        random.randint(2, 10),     # profundidad máxima
        random.randint(2, 10)      # numero de muestras
    ]

def fitness(chrom):
    n_estimators, max_depth, min_samples_split = chrom
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    clf.fit(Xtr, ytr)
    return clf.score(Xval, yval)

def tournament_selection(pop, fitnesses, k=3):
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(range(len(pop))), k)
        winner = max(aspirants, key=lambda i: fitnesses[i])
        selected.append(pop[winner])
    return selected

def one_point_crossover(p1, p2):
    point = random.randint(1, len(p1)-1)
    return p1[:point]+p2[point:], p2[:point]+p1[point:]

def mutate(chrom, pm=0.3):
    if random.random() < pm:
        chrom[0] = random.randint(10,200)
    if random.random() < pm:
        chrom[1] = random.randint(2,10)
    if random.random() < pm:
        chrom[2] = random.randint(2,10)
    return chrom


In [ ]:
POP_SIZE = 6
GENERATIONS = 5

pop = [random_chrom() for _ in range(POP_SIZE)]
print("Población inicial:", pop)

fitnesses = [fitness(ch) for ch in pop]

for gen in range(GENERATIONS):
    print(f"\n=== Generación {gen} ===")
    for i, (chrom, fit) in enumerate(zip(pop, fitnesses)):
        print(f" Individuo {i}: {chrom} → Fitness={fit:.4f}")

    parents = tournament_selection(pop, fitnesses, k=3)

    next_pop = []
    for i in range(0, POP_SIZE, 2):
        c1, c2 = one_point_crossover(parents[i], parents[(i+1)%POP_SIZE])
        next_pop += [c1, c2]

    next_pop = [mutate(c) for c in next_pop]
    print("   Después de mutación:", next_pop)

    fitnesses = [fitness(ch) for ch in next_pop]
    pop = next_pop

best_idx = max(range(len(fitnesses)), key=lambda i: fitnesses[i])
print("\n=== Mejor solución encontrada ===")
print("Mejores hiperparámetros:", pop[best_idx])
print("Fitness:", fitnesses[best_idx])

Población inicial: [[180, 9, 10], [160, 2, 10], [110, 3, 5], [45, 5, 10], [53, 2, 5], [130, 6, 5]]

=== Generación 0 ===
 Individuo 0: [180, 9, 10] → Fitness=1.0000
 Individuo 1: [160, 2, 10] → Fitness=1.0000
 Individuo 2: [110, 3, 5] → Fitness=1.0000
 Individuo 3: [45, 5, 10] → Fitness=1.0000
 Individuo 4: [53, 2, 5] → Fitness=1.0000
 Individuo 5: [130, 6, 5] → Fitness=1.0000
   Después de mutación: [[130, 2, 5], [117, 3, 5], [173, 6, 5], [53, 10, 5], [160, 5, 10], [45, 2, 10]]

=== Generación 1 ===
 Individuo 0: [130, 2, 5] → Fitness=1.0000
 Individuo 1: [117, 3, 5] → Fitness=1.0000
 Individuo 2: [173, 6, 5] → Fitness=1.0000
 Individuo 3: [53, 10, 5] → Fitness=1.0000
 Individuo 4: [160, 5, 10] → Fitness=1.0000
 Individuo 5: [45, 2, 10] → Fitness=1.0000
   Después de mutación: [[13, 9, 5], [173, 10, 5], [130, 2, 5], [162, 3, 5], [117, 3, 9], [160, 5, 5]]

=== Generación 2 ===
 Individuo 0: [13, 9, 5] → Fitness=1.0000
 Individuo 1: [173, 10, 5] → Fitness=1.0000
 Individuo 2: [130, 2, 5

# **NEUROEVOLUTION**

algoritmo evolutivo

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Dataset
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)

# Procesar imágenes
TAMANO_IMG=100 #........................... escala y tamaño.....................
datos_entrenamiento = []
for i, (imagen, etiqueta) in enumerate(datos['train']):
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1)
  datos_entrenamiento.append([imagen, etiqueta])

X = np.array([i[0] for i in datos_entrenamiento]).astype(float)/255
y = np.array([i[1] for i in datos_entrenamiento])

X = X[:2000] # recorte y prueba de redes más rápido
y = y[:2000]

print("Shape:", X.shape, "Labels:", y.shape)


Shape: (2000, 100, 100, 1) Labels: (2000,)


In [ ]:
import random
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def map_lr(idx):
    return {0:1e-2, 1:1e-3, 2:1e-4}[idx]

# Cromosoma = [f1, f2, dense_units, dropout_idx, lr_idx] // cromosoma representando una red con genes:
def build_model_from_chromosome(chrom, input_shape=(100,100,1)): #construcción del modelo
    f1, f2, dense_units, dropout_idx, lr_idx = chrom
    model = Sequential()
    model.add(Conv2D(filters=f1, kernel_size=(3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(filters=f2, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    if dropout_idx == 1:
        model.add(Dropout(0.25))
    elif dropout_idx == 2:
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    lr = map_lr(lr_idx)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
#función de aptitud = val_accuracy //  redes => mejores y descartes
def fitness(chrom):
    model = build_model_from_chromosome(chrom)
    history = model.fit(X, y, batch_size=32,
                        validation_split=0.2,
                        epochs=3,   #pocas epochs para pruebas
                        verbose=0)
    return history.history['val_accuracy'][-1]


In [ ]:
#operadores genéticos
f_choices = [16, 32, 64]
dense_choices = [32, 64, 128]
dropout_idx = [0,1,2]   # 0=no, 1=0.25, 2=0.5
lr_idx = [0,1,2]

def random_chrom():
    return [
        random.choice(f_choices),
        random.choice(f_choices),
        random.choice(dense_choices),
        random.choice(dropout_idx),
        random.choice(lr_idx)
    ]

def tournament_selection(pop, fitnesses, k=3): #mejores individuos para rep.
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(range(len(pop))), k)
        winner = max(aspirants, key=lambda i: fitnesses[i])
        selected.append(pop[winner])
    return selected

def two_point_crossover(p1, p2):               # 2 redes se combinan para nuevas arq.
    a, b = sorted(random.sample(range(len(p1)), 2))
    c1 = p1[:a] + p2[a:b] + p1[b:]
    c2 = p2[:a] + p1[a:b] + p2[b:]
    return c1, c2

def mutate(chrom, pm=0.2):                     # cambiar aleatoriamente un parametro
    if random.random() < pm:
        i = random.randrange(len(chrom))
        chrom[i] = random_chrom()[i]
    return chrom


In [ ]:
POP_SIZE = 4   # pobración de 4 cromos
GENERATIONS = 3

pop = [random_chrom() for _ in range(POP_SIZE)] # gen crom alet con los parametros del CNN
fitnesses = [fitness(ch) for ch in pop]

for gen in range(GENERATIONS):
    print(f"Generación {gen} | Mejor fitness: {max(fitnesses):.4f}")
    parents = tournament_selection(pop, fitnesses, k=3)
    next_pop = []
    for i in range(0, POP_SIZE, 2):
        c1, c2 = two_point_crossover(parents[i], parents[(i+1)%POP_SIZE])
        next_pop += [c1, c2]
    next_pop = [mutate(c) for c in next_pop]
    fitnesses = [fitness(ch) for ch in next_pop]
    pop = next_pop

best_idx = np.argmax(fitnesses)
print("Mejor cromosoma:", pop[best_idx], "con fitness:", fitnesses[best_idx])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Generación 0 | Mejor fitness: 0.6700
Generación 1 | Mejor fitness: 0.6725
Generación 2 | Mejor fitness: 0.6575
Mejor cromosoma: [32, 32, 128, 0, 1] con fitness: 0.6825000047683716


* 32 = filtros en la 1ra capa convolucional.
* 32 = filtros en la 2da capa convolucional.
* 128 = neuronas en la capa densa.
* 0 = dropout = sin dropout.
* 1 = tasa de aprendizaje (learning rate) = 1e-3.

* Fitness 0.6825 = la red alcanzó 68.25% de exactitud en validación.